In [4]:
#364130 to ...
BLOCKS = [
    '000000000000000004e0bb88c983f2c0e812ef2e637bd3ddd48691fb79aad84d',
    '000000000000000002a54709faeda19a20bbcfc4471647238ca98e1d5ef8c66f',
    '0000000000000000092b566770549866470676df750e6a10340ceebe5667d9d8',
    '0000000000000000053197f9e8e5b0601071be99ca2a5c6ba18252a1aa895b04',
    '000000000000000000d54dfc07949110678c611c0c566159d63b70e2496ba93b',
    '00000000000000000d18ced8f58c2d032a6f46bc5c37ebaf80b3ee9fe0776050',
    '00000000000000000e8b99c2bf187fa552c8bdf996c2f5bf1a05f5c6402c742c',
    '0000000000000000020f655cb7c0792906e4cebb54ea834cc6062af12f0563ab'
]

data = []

In [5]:
for BLOCK_HASH in BLOCKS:
    import urllib.request
    block_info = urllib.request.urlopen("https://blockchain.info/rawblock/" + BLOCK_HASH)
    data.append(block_info)

In [6]:
import pickle
pickle.dump( data, open( "bunch_blocks.p", "wb" ) )

In [7]:
import json

In [8]:
data_to_plot = []
for block_info in data:
    block_info_json = json.load(block_info)
    
    nodes_input = []
    nodes_transaction = []
    nodes_output = []
    links = []
    for transaction in block_info_json['tx']:
        nodes_transaction.append(transaction['hash'])
        for input in transaction['inputs']:
            if 'prev_out' in input:
                prev_out = input['prev_out']
                if 'addr' in prev_out:
                    address = prev_out['addr']
                    if address not in nodes_input:
                        nodes_input.append(address)
                    links.append({
                        'source':address,
                        'target':transaction['hash'],
                        'value':1
                    })

        for output in transaction['out']:
            if 'addr' in output:
                address_out = output['addr']
                if address_out not in nodes_output and address_out not in nodes_input:
                    nodes_output.append(address_out)  
                links.append({
                    'source':transaction['hash'],
                    'target':address_out,
                    'value':1
                })
                
    nodes_json = [{'id':a,'group':1} for a in nodes_input] + \
        [{'id':a,'group':2} for a in nodes_transaction] + \
        [{'id':a,'group':3} for a in nodes_output]
    links_json = links
    output = {
        'nodes':nodes_json,
        'links':links
    }
    data_to_plot.append(output)

In [9]:
from selenium import webdriver
from time import sleep

for index,graph in enumerate(data_to_plot):
    with open('20nov.json', 'w') as outfile:
        json.dump(graph, outfile)
    browser = webdriver.Chrome()
    browser.implicitly_wait(1)
    browser.get('about:blank')
    browser.maximize_window()
    sleep(1)
    browser.save_screenshot('blank.png')
    browser.set_window_position(0, 0)
    browser.set_window_size(1000, 1000)
    browser.get('http://localhost:8002/code/20nov.html')
    sleep(80)
    browser.save_screenshot(str(index)+'.png')
    browser.quit()